In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Загрузка данных
df = pd.read_csv('data/root_cause_analysis.csv')

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['ROOT_CAUSE'] = le.fit_transform(data['ROOT_CAUSE'])

# Подготовка данных
X = df.drop(['ID', 'ROOT_CAUSE'], axis=1).values
y = df['CPU_LOAD'].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Преобразование данных в тензоры PyTorch
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()

# Определение сверточной нейронной сети
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(768, 32)
        self.fc2 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x


# Обучение сверточной нейронной сети
net = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
for epoch in range(100):
    optimizer.zero_grad()
    outputs = net(X_train.unsqueeze(1))
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print('Epoch {}, Loss: {:.4f}'.format(epoch, loss.item()))

# Оценка качества модели
y_pred = net(X_test.unsqueeze(1)).detach().numpy()
r2 = r2_score(y_test, y_pred)
print('R-squared:', r2)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (800x32 and 768x32)